In [51]:
#Importing essential libraries
import pandas as pd
import numpy as np
import csv
import json
from itertools import islice
from collections import OrderedDict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import nltk
from glob import glob
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from tqdm import tqdm
import pickle
import math
import re
from sklearn.model_selection import train_test_split
import operator
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')
! pip install git+https://github.com/LIAAD/yake
import yake
! pip install multi_rake
from multi_rake import Rake
! pip install summa
from summa import keywords as summa_keywords
! pip install keybert
from keybert import KeyBERT

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
  Cloning https://github.com/LIAAD/yake to /tmp/pip-req-build-i0en_fj7
  Running command git clone -q https://github.com/LIAAD/yake /tmp/pip-req-build-i0en_fj7
     |████████████████████████████████| 79 kB 6.7 MB/s 
     |████████████████████████████████| 229 kB 49.0 MB/s 
     |████████████████████████████████| 51 kB 6.4 MB/s 
     |████████████████████████████████| 4.0 MB 46.4 MB/s 
     |████████████████████████████████| 1.2 MB 45.6 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
     |████████████████████████████████| 895 kB 44.2 MB/s 
     |████████████████████████████████| 596 kB 40.5 MB/s 
     |████████████████████████████████| 6.6 MB 47.2 MB/s 
  Created wheel for keyb

In [2]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
file1 = open("/content/drive/MyDrive/Tweelink_Dataset/twitter_base_preprocessed.pkl", "rb")
df = pickle.load(file1)
file1.close()

In [5]:
df.head()

,author id,created_at,geo,country,country_code,place_full_name,place_name,place_type,id,lang,...,source,tweet,hashtags,sensitive,urls,context_text,context_probability,context_type,Preprocessed_Data,Date_Only
0,1952510090.0,2022-02-14 00:11:44+00:00,,,,,,,1493015060943454208.0,en,...,Twitter for Android,ð¥ð¥ð¥ð¥ð¥ð¥ð¥â¤µï¸â¤µï¸â¤µï¸...,"CommonSenseGunLaws,GunControlNow,GunSafes,GunS...",False,https://t.co/LjkEup24Dk,,0.0,,[ð¥ð¥ð¥ð¥ð¥ð¥ð¥â¤µï¸â¤µï¸â¤µï¸...,2022-02-14
1,20453105.0,2022-02-14 00:04:56+00:00,,,,,,,1493013352938934272.0,en,...,Twitter for Android,"It's too bad these guys are afraid of needles,...","ClownConvoy,FreeDumbConvoy,OttawaOccupied",False,https://t.co/MmFzuFjIDR,,0.0,,"[bad, guy, afraid, needle, twinrix, would, pre...",2022-02-14
2,1492660754998247424.0,2022-02-14 00:39:55+00:00,,,,,,,1493022155239534336.0,en,...,Twitter for Android,Cowboy dressed as #Furries now available at th...,"Furries,RamRanch,OttawaOccupied",False,https://t.co/GBuBCUtpXe,,0.0,,"[cowboy, dressed, furries, available, ramranch...",2022-02-14
3,117812637.0,2022-02-14 00:20:05+00:00,,,,,,,1493017163485044736.0,en,...,Twitter for iPhone,We blocked these trucks from entering the dntn...,"Riverside,OttawaOccupied",False,https://t.co/KzLkBZvjgD,,0.0,,"[blocked, truck, entering, dntn, core, riversi...",2022-02-14
4,1490044524604928000.0,2022-02-14 00:10:26+00:00,,,,,,,1493014734903382016.0,en,...,Twitter for iPhone,Krista is very pleased with how the @RCMPONT r...,"FluTruxKlanGoHome,OttawaOccupied,kkkonvoy,Otta...",False,https://t.co/kAm5KNugdA,,0.0,,"[krista, pleased, rcmpont, responded, maskless...",2022-02-14


In [6]:
u_base_hashtag = input("Enter base hashtag: ")
u_time = input("Enter time: ")
u_location = input("Enter Location: ")

Enter base hashtag: hijab
Enter time: 2022-02-19
Enter Location: india


In [7]:
import datetime
tweet_query = []
format = '%Y-%m-%d'
u_present_date = datetime.datetime.strptime(u_time, format)
u_prev_date = u_present_date - datetime.timedelta(days=1)
u_next_date = u_present_date + datetime.timedelta(days=1)
df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

In [8]:
df_query.head()

,author id,created_at,geo,country,country_code,place_full_name,place_name,place_type,id,lang,...,source,tweet,hashtags,sensitive,urls,context_text,context_probability,context_type,Preprocessed_Data,Date_Only
12546,123851279.0,2022-02-19 12:59:42+00:00,,,,,,,1495020266862174208.0,en,...,Twitter for iPhone,They are so desperate they have let loose #Yat...,"YatiNarsinghanand,PlotToKillModi,hijab",False,,,0.0,,"[desperate, let, loose, yatinarsinghanand, vio...",2022-02-19
8192,3187630801,2022-02-20 00:55:03+00:00,,,,,,,1495200290210910209,en,...,Twitter for Android,No Woman is Forced to wear and Not to wear Som...,"hijab,HijabIsOurPride,HijabIsFundamentalRight,...",False,,,0.0,,"[woman, forced, wear, wear, something, hijab, ...",2022-02-20
8198,2443976946,2022-02-20 00:14:25+00:00,,,,,,,1495190066112651265,en,...,Twitter Web App,Protests in US cities against #Karnataka #hija...,"Karnataka,hijab,Islamophobic,BJP",False,"https://t.co/957kBM8AJT,https://t.co/5gY741DZP...",,0.0,,"[protest, u, city, karnataka, hijab, ban, indi...",2022-02-20
8278,1483569403425959939,2022-02-20 01:06:11+00:00,18810aa5b43e76c7,Verenigde Staten,US,"Dallas, TX",Dallas,city,1495203094224904192,en,...,Twitter for iPhone,Protest by local Dallas Muslims at JFK square ...,hijab,False,https://t.co/KCm413Yb8N,,0.0,,"[protest, local, dallas, muslim, jfk, square, ...",2022-02-20
8280,18931596,2022-02-20 01:55:42+00:00,,,,,,,1495215554508075011,en,...,Twitter Web App,Feel sorry for #zahirawasim. Religion must ma...,"zahirawasim,hijab",False,,,0.0,,"[feel, sorry, zahirawasim, religion, must, mak...",2022-02-20


In [9]:
def keyword_extractor(dataset):
  preprocessed_vocabulary = dict()

  #Converting to lowercase
  def to_lower_case(text):
    text = text.lower()
    return text

  def remove_at_word(text):
    data = text.split()
    data = [d for d in data if d[0]!='@']
    text = ' '.join(data)
    return text

  def remove_hashtag(text):
    data = text.split()
    data = [d if (d[0]!='#' or len(d) == 1) else d[1:] for d in data]
    data = [d for d in data if d[0]!='#']
    text = ' '.join(data)
    return text

  def remove_URL(text):
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'bit.ly\S+', '', text, flags=re.MULTILINE)
    return text

  #Removing stopwords
  def remove_stopwords(text):
    stopword = stopwords.words('english')
    new_list = [x for x in text.split() if x not in stopword]
    return ' '.join(new_list)

  #Removing punctuations
  def remove_punctuations(text):
    punctuations = '''!()-[|]`{};:'"\,<>./?@#$=+%^&*_~'''
    new_list = ['' if x in punctuations else x for x in text.split()]
    new_list_final = []
    for token in new_list:
      new_token=""
      for char in token:
        if(char not in punctuations):
          new_token+=char
      if(len(new_token)!=0):
        new_list_final.append(new_token)
    return ' '.join(new_list_final)

  #Tokenization
  def tokenization(text):
    return word_tokenize(text)

  def pre_process(text):
    text = to_lower_case(text)
    text = remove_at_word(text)
    text = remove_hashtag(text)
    text = remove_URL(text)
    text = remove_stopwords(text)
    text = remove_punctuations(text)
    text = tokenization(text)
    for token in text:
      if token in preprocessed_vocabulary.keys():
        preprocessed_vocabulary[token] += 1
      else:
        preprocessed_vocabulary[token] = 1
    return text
  
  preprocessed_data = [pre_process(text) for text in dataset]

  #print(preprocessed_vocabulary)

  AOF_coefficient = sum(preprocessed_vocabulary.values())/len(preprocessed_vocabulary)
  vocabulary = {token.strip():preprocessed_vocabulary[token] for token in preprocessed_vocabulary.keys() if preprocessed_vocabulary[token] > AOF_coefficient and len(token.strip())}

  #print(vocabulary)

  final_tokens_per_tweet = []
  for data in preprocessed_data:
    final_tokens_per_tweet.append([token for token in data if token in vocabulary.keys()])

  #print(preprocessed_data)
  #print(final_tokens_per_tweet)

  word2id = dict()
  id2word = dict()
  vocabulary_size = len(vocabulary)
  count = 0
  for token in vocabulary.keys():
    word2id[token] = count
    id2word[count] = token
    count += 1

  #print(word2id)
  #print(id2word)

  directed_graph_adjacency_matrix = np.zeros((vocabulary_size, vocabulary_size))
  edge_weight_matrix = np.zeros((vocabulary_size, vocabulary_size))
  first_frequency = dict()
  last_frequency = dict()
  term_frequency = vocabulary
  strength = dict()
  degree = dict()
  selective_centraility = dict()


  for tweet in final_tokens_per_tweet:

    if tweet[0] in first_frequency.keys():
      first_frequency[tweet[0]] += 1
    else:
      first_frequency[tweet[0]] = 1

    if tweet[-1] in last_frequency.keys():
      last_frequency[tweet[-1]] += 1
    else:
      last_frequency[tweet[-1]] = 1
    


    for i in range(len(tweet)-1):
      if tweet[i] == tweet[i+1]:
        continue
      x = word2id[tweet[i]]
      y = word2id[tweet[i+1]]
      directed_graph_adjacency_matrix[x][y] += 1

  for tweet in final_tokens_per_tweet:
    for i in range(len(tweet)-1):


      if tweet[i] == tweet[i+1]:
        continue
      x = word2id[tweet[i]]
      y = word2id[tweet[i+1]]

    # Updating degree..
      if tweet[i] in degree.keys():
        degree[tweet[i]] += 1
      else:
        degree[tweet[i]] = 1
        
      if tweet[i+1] in degree.keys():
        degree[tweet[i+1]] += 1
      else:
        degree[tweet[i+1]] = 1

      edge_weight_matrix[x][y] = directed_graph_adjacency_matrix[x][y]/(vocabulary[tweet[i]] + vocabulary[tweet[i+1]] - directed_graph_adjacency_matrix[x][y])

      if tweet[i] in strength.keys():
        strength[tweet[i]] += edge_weight_matrix[x][y]
      else:
        strength[tweet[i]] = edge_weight_matrix[x][y]




  first_frequency = {token:(first_frequency[token]/vocabulary[token] if token in first_frequency else 0) for token in vocabulary.keys()}
  last_frequency = {token:(last_frequency[token]/vocabulary[token] if token in last_frequency else 0) for token in vocabulary.keys()}
  degree = {token:(degree[token] if token in degree else 0) for token in vocabulary.keys()}
  strength = {token:(strength[token] if token in strength else 0) for token in vocabulary.keys()}
  selective_centraility = {token:(strength[token]/degree[token] if degree[token]!=0 else 0) for token in vocabulary.keys()}

  #print(degree)
  #print(vocabulary)

  maxdegree = max(degree.items(), key=lambda x: x[1])[1]
  max_degree_nodes_with_freq = {key:term_frequency[key] for key in degree.keys() if degree[key] == maxdegree}
  maxfreq = max(max_degree_nodes_with_freq.items(), key=lambda x: x[1])[1]
  central_node_name = [key for key in max_degree_nodes_with_freq.keys() if max_degree_nodes_with_freq[key] == maxfreq][0]
  #print("central node: ", central_node_name)

  # bfs
  distance_from_central_node = dict()
  central_node_id = word2id[central_node_name]
  q = [(central_node_id, 0)]

  # Set source as visited
  distance_from_central_node[central_node_name] = 0

  while q:
      vis = q[0]
      # Print current node
      #print(id2word[vis[0]], vis[1])
      q.pop(0)
        
      # For every adjacent vertex to
      # the current vertex
      for i in range(len(directed_graph_adjacency_matrix[vis[0]])):
          if (directed_graph_adjacency_matrix[vis[0]][i] == 1 and (id2word[i] not in distance_from_central_node.keys())):
              # Push the adjacent node
              # in the queue
              q.append((i, vis[1]+1))
              distance_from_central_node[id2word[i]] = vis[1]+1

  #print(distance_from_central_node)
  inverse_distance_from_central_node = {token:(1/distance_from_central_node[token] if token in distance_from_central_node and token != central_node_name else 0) for token in vocabulary.keys()}
  inverse_distance_from_central_node[central_node_name] = 1.0
  #print(inverse_distance_from_central_node)

  neighbour_importance = dict()

  for i in range(len(directed_graph_adjacency_matrix)):
    neighbours = set()

    # traversing outgoing edges
    for j in range(len(directed_graph_adjacency_matrix)):
      if i == j:
        continue
      if directed_graph_adjacency_matrix[i][j] > 0:
        neighbours.add(j)
    for j in range(len(directed_graph_adjacency_matrix)):
      if i == j:
        continue
      if directed_graph_adjacency_matrix[j][i] > 0:
          neighbours.add(j)
    if len(neighbours) != 0:
      neighbour_importance[id2word[i]] = sum([strength[id2word[j]] for j in neighbours])/len(neighbours)
    else:
      neighbour_importance[id2word[i]] = 0
      
  #print(neighbour_importance)

  unnormalized_node_weight = {node: (first_frequency[node] + last_frequency[node] + term_frequency[node] + selective_centraility[node] + inverse_distance_from_central_node[node] + neighbour_importance[node]) for node in vocabulary.keys()}
  max_node_weight = max(unnormalized_node_weight.items(), key=lambda x: x[1])[1]
  min_node_weight = min(unnormalized_node_weight.items(), key=lambda x: x[1])[1]
  #print("max node weight: ", max_node_weight, "min node weight: ", min_node_weight)
  normalized_node_weight = {node: ((unnormalized_node_weight[node] - min_node_weight)/(max_node_weight - min_node_weight) if max_node_weight != min_node_weight else unnormalized_node_weight[node]) for node in unnormalized_node_weight.keys()}
  #print("Unnormalized score: ", unnormalized_node_weight)
  #print("Normalized score: ", normalized_node_weight)

  damping_factor = 0.85
  relevance_of_node = {node: np.random.uniform(0,1,1)[0] for node in vocabulary.keys()}
  threshold = 0.000000001


  #print(relevance_of_node)

  count = 0
  while True:
    count += 1
    current_relevance_of_node = dict()
    for node in vocabulary.keys():
      outer_sum = 0
      node_idx = word2id[node]
      for j in range(len(directed_graph_adjacency_matrix)):
        if j == node_idx:
          continue
        if directed_graph_adjacency_matrix[j][node_idx] > 0:
          den_sum = 0
          for k in range(len(directed_graph_adjacency_matrix)):
            if k == j:
              continue
            den_sum += directed_graph_adjacency_matrix[j][k]
          outer_sum += ((directed_graph_adjacency_matrix[j][node_idx]/den_sum) * relevance_of_node[id2word[j]])
      current_relevance_of_node[node] = (1-damping_factor)*normalized_node_weight[node] + damping_factor*normalized_node_weight[node]*outer_sum
    

    # checking convergence..
    sq_error = sum([(current_relevance_of_node[node] - relevance_of_node[node])**2 for node in vocabulary.keys()])
    relevance_of_node = current_relevance_of_node
    if sq_error < threshold:
      break

  #print(relevance_of_node)
  #print(count)

  degree_centrality  = {node: 0 for node in vocabulary.keys()}

  if len(directed_graph_adjacency_matrix) > 1:
    for i in range(len(directed_graph_adjacency_matrix)):
      count = 0
      for j in range(len(directed_graph_adjacency_matrix)):
        if i == j:
          continue
        if directed_graph_adjacency_matrix[j][i] > 0:
          count += 1
      degree_centrality[id2word[i]] = count / (len(directed_graph_adjacency_matrix)-1)

  #print(degree_centrality)

  final_keyword_rank = [{'node': node, 'NE_rank': relevance_of_node[node], 'Degree': degree_centrality[node]} for node in vocabulary.keys()]

  #print("-----------")
  final_keyword_rank = sorted(final_keyword_rank, key = lambda i: (i['NE_rank'], i['Degree']), reverse = True)

  final_keywords = [keyword['node'] for keyword in final_keyword_rank]

  return final_keywords

In [10]:
for tweet in df_query['Preprocessed_Data']:
  tweet_query.extend(tweet)

In [11]:
tweet_query

['desperate',
 'let',
 'loose',
 'yatinarsinghanand',
 'violent',
 'rhetoric',
 'plottokillmodi',
 'hijab',
 'hate',
 'hateâ\x80¦',
 'yatinarsinghanand',
 'plottokillmodi',
 'hijab',
 'woman',
 'forced',
 'wear',
 'wear',
 'something',
 'hijab',
 'hijabisourpride',
 'hijabisfundamentalright',
 'hijaboruniform',
 'hijabcontroversy',
 'hijabcircular',
 'hijabisindividualright',
 'ð\x9f§\x95',
 'hijab',
 'hijabisourpride',
 'hijabisfundamentalright',
 'hijaboruniform',
 'hijabcontroversy',
 'hijabcircular',
 'hijabisindividualright',
 'protest',
 'u',
 'city',
 'karnataka',
 'hijab',
 'ban',
 'indian',
 'american',
 'ally',
 'demonstrated',
 'islamophobic',
 'unconstitutional',
 'ban',
 'school',
 'india',
 's',
 'nationalist',
 'bjp',
 'government',
 'take',
 'action',
 'canadaâ\x9e¡ï¸\x8f',
 'http',
 'tco957kbm8ajt',
 'take',
 'action',
 'globallyâ\x9e¡ï¸\x8f',
 'http',
 'tco5gy741dzp1',
 'http',
 'tcobvsjhjohkc',
 'karnataka',
 'hijab',
 'islamophobic',
 'bjp',
 'protest',
 'local',
 '

In [12]:
keyword_dataset = df_query['tweet'].tolist()
tweet_query_keyword_extractor = keyword_extractor(keyword_dataset)

In [13]:
tweet_query_keyword_extractor

['hijab',
 'muslim',
 'women',
 'india',
 'wear',
 'muslims',
 'hijabrow',
 'islam',
 'allah',
 'like',
 'girls',
 'hijabisourright',
 'karnataka',
 'wearing',
 'amp',
 'school',
 'hijabisourpride',
 'ban',
 'you',
 'hijabisfundamentalright',
 'girl',
 'hijabcontroversy',
 'one',
 'education',
 'choice',
 'need',
 'world',
 'schools',
 'people',
 'islamic',
 'woman',
 'also',
 'want',
 'religion',
 'first',
 'bjp',
 'right',
 'even',
 'sitting',
 'hijabisindividualright',
 'leaders',
 'judge',
 'hijabban',
 'protest',
 'quietly',
 'students',
 'uniform',
 'tell',
 'allowed',
 'fighting',
 'colleges',
 'burqa',
 'issue',
 'started',
 'surely',
 'public',
 'decision',
 'religious',
 'said',
 'support',
 'indian',
 'rely',
 'alive',
 'abaya',
 'islamophobia',
 'freedom',
 'muslimah',
 'persecution',
 'allahuakbar',
 'sunnah',
 'see',
 'ramadan',
 'makkah',
 'alhamdulillah',
 'get',
 'islamicquotes',
 'dua',
 'quran',
 'rights',
 'hijabplot',
 'country',
 'hindu',
 'practice',
 'remove',
 

In [14]:
tweet_keywords_yake = []
kw_extractor = yake.KeywordExtractor(top=20, stopwords=None)
keywords = kw_extractor.extract_keywords(' '.join(tweet_query))
#keywords = kw_extractor.extract_keywords(' '.join(df_query['tweet'].tolist()))
for kw, v in keywords:
  #print("Keyphrase: ",kw, ": score", v)
  for key in kw.split():
    if(key.lower() not in tweet_keywords_yake):
      tweet_keywords_yake.append(key.lower())

print(tweet_keywords_yake)


Keyphrase:  hijab hijab hijab : score 3.538515818965514e-07
Keyphrase:  hijab karnataka india : score 4.4295050053984284e-07
Keyphrase:  muslim islam muslim : score 7.831784725969966e-07
Keyphrase:  hijab hijabisfundamentalright india : score 8.733034882458195e-07
Keyphrase:  hijabisourright hijabcontroversy hijab : score 1.2084471666090418e-06
Keyphrase:  karnataka india hijab : score 1.2402614015115601e-06
Keyphrase:  india hijab karnataka : score 1.2402614015115601e-06
Keyphrase:  muslim hijab started : score 1.3280528877404291e-06
Keyphrase:  hijab abaya muslim : score 1.3444833831980227e-06
Keyphrase:  hijab hijabrow http : score 1.425915298203952e-06
Keyphrase:  persecution muslim hijab : score 1.4602327688888964e-06
Keyphrase:  hijabcontroversy hijab hijabisfundamentalright : score 1.5153685192729542e-06
Keyphrase:  hijabisourright hijab http : score 1.6067046974268703e-06
Keyphrase:  wear hijab hijab : score 1.6311894877235963e-06
Keyphrase:  hijab muslim india : score 1.643581

In [45]:
tweet_keywords_rake = []
rake = Rake()
rake_keywords = rake.apply(' '.join(tweet_query).encode('ascii', 'ignore').decode())
for kw,score in rake_keywords[:20]:
  for key in kw.split():
    if(key.lower() not in tweet_keywords_yake):
      tweet_keywords_rake.append(key.lower())
print(tweet_keywords_rake)

['nationalist', 'bjp', 'government', 'puppet', 'political', 'people', 'remove', '1st', 'sanghis', 'carrying', 'religious', 'commitment', 'uk', 'usa', 'dude', 'destruction', 'implosion', 'fear', 'side', 'coin', 'matter', 'member', 'faith', 'bradford', 'bid', 'scaleup', 'tourism', 'shawl', 'order', 'visit', 'understand', 'prince', 'respect', 'part', 'dress', 'code', 'sexually', 'abusing', '9nyear', 'upper', 'caste', 'idiot', 'allowed', 'wait', 'god', 'vote', 'bjp', 'understand', 'kasuwa24', 'youre', 'click', 'subahanahuwataala', 'bless', 'issue', 'claim', 'pardah', 'varies']


In [49]:
tweet_keywords_text_rank = []
TR_keywords = summa_keywords.keywords(' '.join(tweet_query), scores=True)
for kw,score in TR_keywords[:20]:
  for key in kw.split():
    if(key.lower() not in tweet_keywords_text_rank):
      tweet_keywords_text_rank.append(key.lower())
print(tweet_keywords_text_rank)

['hijabers', 'http', 'hijab', 'hate', 'ð', 'â', 'school', 'india', 'wearing', 'girls', 'woman', 'forced', 'wear', 'hijabrow', 'amp', 'islamic', 'local', 'dallas', 'muslim', 'tco', 'educated', 'like', 'banning', 'banned', 'girl', 'suddenly', 'wanted', 'bjp']


In [53]:
keybert_model = KeyBERT(model='all-mpnet-base-v2')
keybert_keywords = keybert_model.extract_keywords(' '.join(tweet_query), keyphrase_ngram_range=(1,1), stop_words='english', highlight=False, top_n=20)
tweet_keywords_keybert = list(dict(keybert_keywords).keys())
print(tweet_keywords_keybert)

['hijabisfundamentalright', 'hijabers', 'hijabkarnataka', 'hijab', 'hijabi', 'hijabifashion', 'hijabday', 'hijabisourright', 'hijabporn', 'hijabisindividualright', 'hijabfashion', 'hijaband', 'hijabban', 'hijabislut', 'hijabisouridentity', 'hijabismyright', 'hijabð', 'hijabis', 'hijaab', 'burkah']


In [15]:
docs_preprocessed = []

In [16]:
#Storing file name and data
total_documents = 0
path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
for filename in glob(os.path.join(path, '*')):
   with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
     filename = os.path.basename(f.name)
     data = json.load(f)
     d_date = data["Date"]
     if(d_date=="" or d_date=="Date"):
       continue
     format = '%Y-%m-%d'
 
     d_present_date = datetime.datetime.strptime(d_date, format)
 
     if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
       continue
   
     docs_preprocessed.append({'Name':filename, 'Data':data})
     total_documents+=1
print(total_documents)

295


In [17]:
def get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed):
  relevant_docs_list = []
  for doc in docs_preprocessed:
    if doc['Data']['Base Hashtag']==base_hashtag:
      current_date = datetime.datetime.strptime(base_date, format)
      prev_date = current_date - datetime.timedelta(days=1)
      next_date = current_date + datetime.timedelta(days=1)
      if(doc['Data']['Date'] in [str(prev_date.date()), str(current_date.date()), str(next_date.date())]):
        relevant_docs_list.append(doc['Name'])
  return relevant_docs_list

In [18]:
def precision_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr][0] in relevant_docs_list):
      num_of_relevant_results+=1
  return num_of_relevant_results/k

In [19]:
def mean_average_precision(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_precision=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    precision_at_k_val = precision_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    #print('Hashtag: {}   Precision@{}: {}'.format(base_hashtag, k_val, precision_at_k_val))
    average_precision += precision_at_k_val
  return average_precision/ctr

In [20]:
def recall_at_k(k, base_hashtag, base_date, prediction_list, docs_preprocessed):
  relevant_docs_list = get_relevant_docs_list_for_base_hashtag(base_hashtag, base_date, docs_preprocessed)
  current_num_of_relevant_results=0
  for itr in range(k):
    if (prediction_list[itr][0] in relevant_docs_list):
      current_num_of_relevant_results+=1
  if(len(relevant_docs_list)==0):
    return 0
  return current_num_of_relevant_results/len(relevant_docs_list)

In [21]:
def mean_average_recall(max_k, base_hashtag, base_date, relevant_docs, docs_preprocessed):
  average_recall=0
  ctr=0
  for k_val in range(1,max_k+1):
    ctr+=1
    recall_at_k_val = recall_at_k(k_val, base_hashtag, base_date, relevant_docs, docs_preprocessed)
    #print('Hashtag: {}   Recall@{}: {}'.format(base_hashtag, k_val, recall_at_k_val))
    average_recall += recall_at_k_val
  return average_recall/ctr

In [22]:
import operator

def find_relevant_documents(docs_preprocessed, processed_query):
  jaccard_coefficients = {}
  for document in docs_preprocessed:
    doc_text = set(document['Data']['Body_processed'])
    query = set(processed_query)
    jaccard_coefficients[document['Name']] = len(doc_text.intersection(query))/len(doc_text.union(query))
  relevant_docs = list( sorted(jaccard_coefficients.items(), key=operator.itemgetter(1),reverse=True))[:20]
  for i in range(len(relevant_docs)):
    for j in range(len(docs_preprocessed)):
      if(relevant_docs[i][0] == docs_preprocessed[j]['Name']):
        relevant_docs[i] = (relevant_docs[i][0], relevant_docs[i][1], docs_preprocessed[j]['Data']['Date'] )

  return relevant_docs



Plain Model

In [54]:
# Plain Model without YAKE / Keyword Extraction
relevant_docs_plain = find_relevant_documents(docs_preprocessed, tweet_query)

for rank, doc in enumerate(relevant_docs_plain):
  print('Rank: {} Relevant Document: {}'.format(rank+1,doc))

print()

mean_average_precision_hashtag_plain = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_plain, docs_preprocessed)
print('Mean Average Precision Plain Model: {}'.format(mean_average_precision_hashtag_plain))

mean_average_recall_hashtag_plain = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_plain, docs_preprocessed)
print('Mean Average Recall Plain Model: {}'.format(mean_average_recall_hashtag_plain))

Rank: 1 Relevant Document: ('WriddhimanSaha_133.json', 0.08778625954198473, '2022-02-20')
Rank: 2 Relevant Document: ('PunjabElections2022_127.json', 0.08217317487266554, '2022-02-20')
Rank: 3 Relevant Document: ('PunjabElections2022_124.json', 0.06238587948874011, '2022-02-20')
Rank: 4 Relevant Document: ('UkraineRussiaCrisis_220.json', 0.059113300492610835, '2022-02-18')
Rank: 5 Relevant Document: ('vaccine_299.json', 0.058758314855875834, '2022-02-19')
Rank: 6 Relevant Document: ('UkraineRussiaCrisis_212.json', 0.05858511422254974, '2022-02-18')
Rank: 7 Relevant Document: ('MultiverseOfMadness_16.json', 0.058354476259514315, '2022-02-18')
Rank: 8 Relevant Document: ('BJPwinningUP_63.json', 0.057449253159708925, '2022-02-18')
Rank: 9 Relevant Document: ('OperationDudula_261.json', 0.05601503759398496, '2022-02-20')
Rank: 10 Relevant Document: ('ScottyFromWelding_170.json', 0.054324517512508934, '2022-02-20')
Rank: 11 Relevant Document: ('UkraineRussiaCrisis_261.json', 0.0540057224606

Model with Keyword Extractor

In [55]:
# Model with Keyword Extractor
relevant_docs_keyword_extractor = find_relevant_documents(docs_preprocessed, tweet_query_keyword_extractor[:20])

for rank, doc in enumerate(relevant_docs_keyword_extractor):
  print('Rank: {} Relevant Document: {}'.format(rank+1,doc))

print()

mean_average_precision_hashtag_keyword_extractor = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_keyword_extractor, docs_preprocessed)
print('Mean Average Precision Keyword Extractor Model: {}'.format(mean_average_precision_hashtag_keyword_extractor))

mean_average_recall_hashtag_keyword_extractor = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_keyword_extractor, docs_preprocessed)
print('Mean Average Recall Keyword Extractor Model: {}'.format(mean_average_recall_hashtag_keyword_extractor))

Rank: 1 Relevant Document: ('hijab_288.json', 0.0898876404494382, '2022-02-19')
Rank: 2 Relevant Document: ('hijab_282.json', 0.04046242774566474, '2022-02-19')
Rank: 3 Relevant Document: ('hijab_281.json', 0.03864734299516908, '2022-02-19')
Rank: 4 Relevant Document: ('hijab_290.json', 0.03864734299516908, '2022-02-19')
Rank: 5 Relevant Document: ('hijab_285.json', 0.03501945525291829, '2022-02-19')
Rank: 6 Relevant Document: ('hijab_287.json', 0.03389830508474576, '2022-02-19')
Rank: 7 Relevant Document: ('hijab_283.json', 0.026737967914438502, '2022-02-19')
Rank: 8 Relevant Document: ('hijab_286.json', 0.02622950819672131, '2022-02-19')
Rank: 9 Relevant Document: ('BJPwinningUP_66.json', 0.024844720496894408, '2022-02-18')
Rank: 10 Relevant Document: ('hijab_289.json', 0.023952095808383235, '2022-02-19')
Rank: 11 Relevant Document: ('hijab_284.json', 0.019553072625698324, '2022-02-19')
Rank: 12 Relevant Document: ('WriddhimanSaha_132.json', 0.015625, '2022-02-20')
Rank: 13 Relevant 

Model with YAKE

In [56]:
# Model with YAKE
relevant_docs_yake = find_relevant_documents(docs_preprocessed, tweet_keywords_yake)

for rank, doc in enumerate(relevant_docs_yake):
  print('Rank: {} Relevant Document: {}'.format(rank+1,doc))

print()

mean_average_precision_hashtag_yake = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_yake, docs_preprocessed)
print('Mean Average Precision YAKE Model: {}'.format(mean_average_precision_hashtag_yake))

mean_average_recall_hashtag_yake = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_yake, docs_preprocessed)
print('Mean Average Recall YAKE Model: {}'.format(mean_average_recall_hashtag_yake))

Rank: 1 Relevant Document: ('hijab_288.json', 0.06741573033707865, '2022-02-19')
Rank: 2 Relevant Document: ('hijab_282.json', 0.04093567251461988, '2022-02-19')
Rank: 3 Relevant Document: ('hijab_285.json', 0.03529411764705882, '2022-02-19')
Rank: 4 Relevant Document: ('hijab_290.json', 0.03398058252427184, '2022-02-19')
Rank: 5 Relevant Document: ('hijab_281.json', 0.028985507246376812, '2022-02-19')
Rank: 6 Relevant Document: ('hijab_287.json', 0.028409090909090908, '2022-02-19')
Rank: 7 Relevant Document: ('hijab_283.json', 0.02702702702702703, '2022-02-19')
Rank: 8 Relevant Document: ('narendramodi_255.json', 0.022727272727272728, '2022-02-19')
Rank: 9 Relevant Document: ('IndiaFightsCorona_241.json', 0.020202020202020204, '2022-02-18')
Rank: 10 Relevant Document: ('hijab_286.json', 0.019672131147540985, '2022-02-19')
Rank: 11 Relevant Document: ('BJPwinningUP_66.json', 0.01875, '2022-02-18')
Rank: 12 Relevant Document: ('hijab_289.json', 0.018072289156626505, '2022-02-19')
Rank: 

Model with RAKE

In [57]:
# Model with RAKE
relevant_docs_rake = find_relevant_documents(docs_preprocessed, tweet_keywords_rake)

for rank, doc in enumerate(relevant_docs_rake):
  print('Rank: {} Relevant Document: {}'.format(rank+1,doc))

print()

mean_average_precision_hashtag_rake = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_rake, docs_preprocessed)
print('Mean Average Precision RAKE Model: {}'.format(mean_average_precision_hashtag_rake))

mean_average_recall_hashtag_rake = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_rake, docs_preprocessed)
print('Mean Average Recall RAKE Model: {}'.format(mean_average_recall_hashtag_rake))

Rank: 1 Relevant Document: ('hijab_282.json', 0.043689320388349516, '2022-02-19')
Rank: 2 Relevant Document: ('narendramodi_209.json', 0.041379310344827586, '2022-02-18')
Rank: 3 Relevant Document: ('narendramodi_255.json', 0.040983606557377046, '2022-02-19')
Rank: 4 Relevant Document: ('hijab_289.json', 0.035175879396984924, '2022-02-19')
Rank: 5 Relevant Document: ('hijab_284.json', 0.03359173126614987, '2022-02-19')
Rank: 6 Relevant Document: ('narendramodi_253.json', 0.03333333333333333, '2022-02-19')
Rank: 7 Relevant Document: ('ShivajiJayanti_205.json', 0.03225806451612903, '2022-02-18')
Rank: 8 Relevant Document: ('hijab_283.json', 0.031818181818181815, '2022-02-19')
Rank: 9 Relevant Document: ('hijab_288.json', 0.03125, '2022-02-19')
Rank: 10 Relevant Document: ('ScottyFromWelding_164.json', 0.030864197530864196, '2022-02-20')
Rank: 11 Relevant Document: ('hijab_285.json', 0.030821917808219176, '2022-02-19')
Rank: 12 Relevant Document: ('JohnsonOut21_45.json', 0.030487804878048

Model with TextRank

In [58]:
# Model with TextRank
relevant_docs_text_rank = find_relevant_documents(docs_preprocessed, tweet_keywords_text_rank)

for rank, doc in enumerate(relevant_docs_text_rank):
  print('Rank: {} Relevant Document: {}'.format(rank+1,doc))

print()

mean_average_precision_hashtag_text_rank = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_text_rank, docs_preprocessed)
print('Mean Average Precision TextRank Model: {}'.format(mean_average_precision_hashtag_text_rank))

mean_average_recall_hashtag_text_rank = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_text_rank, docs_preprocessed)
print('Mean Average Recall TextRank Model: {}'.format(mean_average_recall_hashtag_text_rank))

Rank: 1 Relevant Document: ('hijab_288.json', 0.07142857142857142, '2022-02-19')
Rank: 2 Relevant Document: ('BJPwinningUP_66.json', 0.048484848484848485, '2022-02-18')
Rank: 3 Relevant Document: ('hijab_282.json', 0.03867403314917127, '2022-02-19')
Rank: 4 Relevant Document: ('hijab_285.json', 0.03787878787878788, '2022-02-19')
Rank: 5 Relevant Document: ('hijab_281.json', 0.037209302325581395, '2022-02-19')
Rank: 6 Relevant Document: ('hijab_290.json', 0.037209302325581395, '2022-02-19')
Rank: 7 Relevant Document: ('hijab_283.json', 0.03626943005181347, '2022-02-19')
Rank: 8 Relevant Document: ('hijab_286.json', 0.035483870967741936, '2022-02-19')
Rank: 9 Relevant Document: ('hijab_287.json', 0.032432432432432434, '2022-02-19')
Rank: 10 Relevant Document: ('PlotToKillModi_220.json', 0.023121387283236993, '2022-02-18')
Rank: 11 Relevant Document: ('hijab_289.json', 0.022857142857142857, '2022-02-19')
Rank: 12 Relevant Document: ('BoycottWalgreens_249.json', 0.020833333333333332, '2022

Model with KeyBERT

In [59]:
# Model with KeyBERT
relevant_docs_keybert = find_relevant_documents(docs_preprocessed, tweet_keywords_keybert)

for rank, doc in enumerate(relevant_docs_keybert):
  print('Rank: {} Relevant Document: {}'.format(rank+1,doc))

print()

mean_average_precision_hashtag_keybert = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs_keybert, docs_preprocessed)
print('Mean Average Precision KeyBERT Model: {}'.format(mean_average_precision_hashtag_keybert))

mean_average_recall_hashtag_keybert = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs_keybert, docs_preprocessed)
print('Mean Average Recall KeyBERT Model: {}'.format(mean_average_recall_hashtag_keybert))

Rank: 1 Relevant Document: ('hijab_288.json', 0.010416666666666666, '2022-02-19')
Rank: 2 Relevant Document: ('hijab_289.json', 0.0058823529411764705, '2022-02-19')
Rank: 3 Relevant Document: ('hijab_282.json', 0.00558659217877095, '2022-02-19')
Rank: 4 Relevant Document: ('hijab_287.json', 0.005494505494505495, '2022-02-19')
Rank: 5 Relevant Document: ('hijab_283.json', 0.005235602094240838, '2022-02-19')
Rank: 6 Relevant Document: ('hijab_281.json', 0.004672897196261682, '2022-02-19')
Rank: 7 Relevant Document: ('hijab_290.json', 0.004672897196261682, '2022-02-19')
Rank: 8 Relevant Document: ('hijab_285.json', 0.0037735849056603774, '2022-02-19')
Rank: 9 Relevant Document: ('hijab_286.json', 0.003205128205128205, '2022-02-19')
Rank: 10 Relevant Document: ('hijab_284.json', 0.0027472527472527475, '2022-02-19')
Rank: 11 Relevant Document: ('SuperBowl_10.json', 0.0, '2022-02-20')
Rank: 12 Relevant Document: ('MultiverseOfMadness_16.json', 0.0, '2022-02-18')
Rank: 13 Relevant Document: (

**Across all hashtags**

In [27]:
# global_list = [['narendramodi', '2022-02-14', 'india'],['UkraineRussiaCrisis', '2022-02-14', 'ukraine'],['IPL', '2022-02-14', 'india'],['TaylorSwift', '2022-02-14', 'USA'],['IndiaFightsCorona', '2022-02-14', 'india'],['narendramodi', '2022-02-15', 'india'],['UkraineRussiaCrisis', '2022-02-15', 'ukraine'],['IPL', '2022-02-15', 'india'],['TaylorSwift', '2022-02-15', 'USA'],['IndiaFightsCorona', '2022-02-15', 'india'],['narendramodi', '2022-02-16', 'india'],['UkraineRussiaCrisis', '2022-02-16', 'ukraine'],['IPL', '2022-02-16', 'india'],['TaylorSwift', '2022-02-16', 'USA'],['IndiaFightsCorona', '2022-02-16', 'india'],['narendramodi', '2022-02-17', 'india'],['UkraineRussiaCrisis', '2022-02-17', 'ukraine'],['IPL', '2022-02-17', 'india'],['TaylorSwift', '2022-02-17', 'USA'],['IndiaFightsCorona', '2022-02-17', 'india'],['narendramodi', '2022-02-18', 'india'],['UkraineRussiaCrisis', '2022-02-18', 'ukraine'],['IPL', '2022-02-18', 'india'],['TaylorSwift', '2022-02-18', 'USA'],['IndiaFightsCorona', '2022-02-18', 'india'],['narendramodi', '2022-02-19', 'india'],['UkraineRussiaCrisis', '2022-02-19', 'ukraine'],['IPL', '2022-02-19', 'india'],['hijab', '2022-02-19', 'india'],['vaccine', '2022-02-19', 'india'],['MillionAtIndiaPavilion', '2022-02-14', 'UAE'],['PunjabPanjeNaal', '2022-02-14', 'India'],['Euphoria', '2022-02-14', 'World'],['OscarsFanFavorite', '2022-02-14', 'World'],['ShameOnBirenSingh', '2022-02-14', 'india'],['BappiLahiri', '2022-02-16', 'india'],['BlandDoritos', '2022-02-16', 'USA'],['VERZUZ', '2022-02-16', 'USA'],['DragRaceUK', '2022-02-16', 'United Kingdom'],['BoycottWalgreens', '2022-02-18', 'USA'],['PunjabElections2022', '2022-02-20', 'india'],['WriddhimanSaha', '2022-02-20', 'india'],['stormfranklin', '2022-02-20', 'USA'],['QueenElizabeth', '2022-02-20', 'United Kingdom'],['ScottyFromWelding', '2022-02-20', 'Australia'],['CarabaoCupFinal', '2022-02-27', 'London'],['NZvSA', '2022-02-28', 'New Zealand'],['IPCC', '2022-02-28', 'Worldwide'],['SuperBowl', '2022-02-14', 'USA'],['MultiverseOfMadness', '2022-02-14', 'USA'],['Eminem', '2022-02-14', 'USA'],['IPLAuction', '2022-02-14', 'india'],['JohnsonOut21', '2022-02-14', 'United Kingdom'],['Cyberpunk2077', '2022-02-15', 'Worldwide'],['Wordle242', '2022-02-15', 'Worldwide'],['DeepSidhu', '2022-02-15', 'india'],['CanadaHasFallen', '2022-02-15', 'canada'],['IStandWithTrudeau', '2022-02-15', 'canada'],['CNNPHVPDebate', '2022-02-26', 'philippines'],['qldfloods', '2022-02-26', 'australia'],['Eurovision', '2022-02-26', 'worldwide'],['IndiansInUkraine', '2022-02-26', 'india'],['PritiPatel', '2022-02-26', 'united kingdom'],['TaylorCatterall', '2022-02-27', 'united kingdom'],['PSLFinal', '2022-02-27', 'pakistan'],['AustraliaDecides', '2022-02-27', 'australia'],['WorldNGODay', '2022-02-27', 'worldwide'],['TheBatman', '2022-02-28', 'USA'],['NationalScienceDay', '2022-02-28', 'india'],['msdtrong', '2022-02-14', 'india'],['Boycott_ChennaiSuperKings', '2022-02-14', 'india'],['GlanceJio', '2022-02-14', 'india'],['ArabicKuthu', '2022-02-14', 'india'],['Djokovic', '2022-02-15', 'australia'],['Real Madrid', '2022-02-15', 'santiago'],['bighit', '2022-02-15', 'korea'],['Maxwell', '2022-02-15', 'australia'],['mafsau', '2022-02-16', 'australia'],['channi', '2022-02-16', 'punjab'],['ayalaan', '2022-02-16', 'india'],['jkbose', '2022-02-16', 'india'],['HappyBirthdayPrinceSK', '2022-02-16', 'india'],['RandomActsOfKindnessDay', '2022-02-17', 'worldwide'],['happybirthdayjhope', '2022-02-17', 'korea'],['mohsinbaig', '2022-02-17', 'pakistan'],['aewdynamite', '2022-02-17', 'worldwide'],['aaraattu', '2022-02-17', 'india'],['ShivajiJayanti', '2022-02-18', 'india'],['PlotToKillModi', '2022-02-18', 'india'],['NationalDrinkWineDay', '2022-02-18', 'usa'],['HorizonForbiddenWest', '2022-02-18', 'worldwide'],['BoycottWalgreens', '2022-02-18', 'usa'],['CallTheMidwife', '2022-02-20', 'worldwide'],['OperationDudula', '2022-02-20', 'south africa'],['truthsocial', '2022-02-21', 'usa'],['nbaallstar', '2022-02-21', 'usa'],['shivamogga', '2022-02-21', 'india'],['HalftimeShow', '2022-02-14', 'usa'],['OttawaStrong', '2022-02-14', 'canada'],['DrDre', '2022-02-14', 'usa'],['BattleOfBillingsBridge', '2022-02-14', 'usa'],['FullyFaltooNFTdrop', '2022-02-14', 'worldwide'],['AK61', '2022-02-15', 'india'],['sandhyamukherjee', '2022-02-15', 'india'],['MUNBHA', '2022-02-15', 'worldwide'],['nursesstrike', '2022-02-15', 'australia'],['Realme9ProPlus', '2022-02-16', 'worldwide'],['KarnatakaHijabControversy', '2022-02-16', 'india'],['BJPwinningUP', '2022-02-16', 'india'],['Punjab_With_Modi', '2022-02-16', 'india'],['PushpaTheRule', '2022-02-16', 'india'],['RehmanMalik', '2022-02-22', 'india'],['harisrauf', '2022-02-22', 'pakistan'],['Rosettenville', '2022-02-22', 'south africa'],['NFU22', '2022-02-22', 'worldwide'],['justiceforharsha', '2022-02-22', 'india'],['wordle251', '2022-02-24', 'worldwide'],['ARSWOL', '2022-02-24', 'worldwide'],['stopwar', '2022-02-24', 'worldwide'],['PrayForPeace', '2022-02-24', 'worldwide'],['StopPutinNOW', '2022-02-24', 'worldwide'],['TeamGirlsCup', '2022-02-25', 'worldwide'],['Canucks', '2022-02-25', 'worldwide'],['PinkShirtDay', '2022-02-25', 'canada'],['superrugbypacific', '2022-02-25', 'australia']]

In [28]:
# global_average_mean_average_precision = []
# global_mean_average_recall = []

# for iter in tqdm(range(len(global_list))):
#   u_base_hashtag = global_list[iter][0]
#   u_time = global_list[iter][1]
#   u_location = global_list[iter][2]
#   tweet_query = []
#   format = '%Y-%m-%d'
#   u_present_date = datetime.datetime.strptime(u_time, format)
#   u_prev_date = u_present_date - datetime.timedelta(days=1)
#   u_next_date = u_present_date + datetime.timedelta(days=1)
#   df_query = df.loc[df['hashtags'].str.contains(u_base_hashtag) & df['Date_Only'].isin([str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())])]

#   for tweet in df_query['Preprocessed_Data']:
#     tweet_query.extend(tweet)
  
#   tweet_keywords = []
#   kw_extractor = yake.KeywordExtractor(top=20, stopwords=None)
#   keywords = kw_extractor.extract_keywords(' '.join(tweet_query))
#   for kw, v in keywords:
#     #print("Keyphrase: ",kw, ": score", v)
#     for key in kw.split():
#       if(key not in tweet_keywords):
#         tweet_keywords.append(key)
  
#   docs_preprocessed = []

#   total_documents = 0
#   path = '/content/drive/MyDrive/Tweelink_Dataset/Tweelink_Articles_Processed'
#   for filename in glob(os.path.join(path, '*')):
#     with open(os.path.join(os.getcwd(), filename), 'r', encoding = 'utf-8',errors = 'ignore') as f:
#       filename = os.path.basename(f.name)
#       data = json.load(f)
#       d_date = data["Date"]
#       if(d_date=="" or d_date=="Date"):
#         continue
#       format = '%Y-%m-%d'
  
#       d_present_date = datetime.datetime.strptime(d_date, format)
  
#       if(str(d_present_date.date()) not in [str(u_present_date.date()), str(u_prev_date.date()), str(u_next_date.date())]):
#         continue
    
#       docs_preprocessed.append({'Name':filename, 'Data':data})
#       total_documents+=1
  
#   # with yake
#   # relevant_docs = find_relevant_documents(docs_preprocessed, tweet_keywords)

#   # without yake/ keyword extraction
#   relevant_docs = find_relevant_documents(docs_preprocessed, tweet_query)

#   mean_average_precision_hashtag = mean_average_precision(20, u_base_hashtag, u_time, relevant_docs, docs_preprocessed)
#   global_average_mean_average_precision.append(mean_average_precision_hashtag)
#   #print(mean_average_precision_hashtag)

#   mean_average_recall_hashtag = mean_average_recall(20, u_base_hashtag, u_time, relevant_docs, docs_preprocessed)
#   global_mean_average_recall.append(mean_average_recall_hashtag)
#   #print(mean_average_recall_hashtag)

In [29]:
# overall_average_mean_average_precision = sum(global_average_mean_average_precision)/len(global_average_mean_average_precision)
# print(overall_average_mean_average_precision)

In [30]:
# overall_mean_average_recall = sum(global_mean_average_recall)/len(global_mean_average_recall)
# print(overall_mean_average_recall)